The dataset provided for this study delineates the Global Corporate Average One-year Transition Rates Matrix covering the period from 1981 to 2022. This matrix segregates non-defaulting debtors into seven credit ratings: AAA, AA, A, BBB, BB, B, CCC, and adds a distinct classification for defaults, marked as D. Close examination of this matrix suggests its derivation using the cohort method, characterizing it as a discrete migration matrix. A notable characteristic of a discrete migration matrix is the occurrence of zero values in some migration probabilities, implying that the matrix tracks the year-end positions of borrowers rather than their transitions throughout the year. Furthermore, an analysis of the matrix's last column discloses the default probabilities for each credit rating, indicative of the 1-year Probability of Default (PDs).

The Cohort method used in calculation is the industry standard for count data but it also has drawbacks. Any rating change activity which occurs withing the period Δ𝑡 is ignored. In result, this type of discrete migration matrix become unreliable when credits change their rating class frequently.

Beginning with the one-year migration matrix, we can expand it to longer periods like 2 or even 10 years. However, this process is contingent on an important caveat: the discrete migration matrix is not inherently time-homogeneous. Therefore, assuming time-homogeneity becomes a prerequisite before we can utilize the formula needed to derive the discrete migration matrix for these extended periods.

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns

data_mm = pd.read_excel('CaseStudyData_2023-1.xlsx').set_index('Unnamed: 0')
pd.options.display.float_format = '{:,.2f}'.format


# Adjusting Data for further purposes
# Creating migration matrices for year up to 10

# This will further allow to calculate multi-year probability of default for each rating class
markov_1y = data_mm.set_index(data_mm.columns)
markov_2y = pd.DataFrame(np.dot(data_mm, data_mm), columns=data_mm.columns).set_index(data_mm.columns)
markov_3y = pd.DataFrame(np.dot(markov_2y, data_mm), columns=data_mm.columns).set_index(data_mm.columns)
markov_4y = pd.DataFrame(np.dot(markov_3y, data_mm), columns=data_mm.columns).set_index(data_mm.columns)
markov_5y = pd.DataFrame(np.dot(markov_4y, data_mm), columns=data_mm.columns).set_index(data_mm.columns)
markov_6y = pd.DataFrame(np.dot(markov_5y, data_mm), columns=data_mm.columns).set_index(data_mm.columns)
markov_7y = pd.DataFrame(np.dot(markov_6y, data_mm), columns=data_mm.columns).set_index(data_mm.columns)
markov_8y = pd.DataFrame(np.dot(markov_7y, data_mm), columns=data_mm.columns).set_index(data_mm.columns)
markov_9y = pd.DataFrame(np.dot(markov_8y, data_mm), columns=data_mm.columns).set_index(data_mm.columns)
markov_10y = pd.DataFrame(np.dot(markov_9y, data_mm), columns=data_mm.columns).set_index(data_mm.columns)

In [12]:
# Change of visual aspects to better present matrices and Tables
cm = sns.light_palette("blue", as_cmap=True)

The following 1-year migration matrix was scaled to 10 years using the above (see complete report in .pdf) formula. After having a complete migration matrix for each year 1 to 10, it is possible to calculate the multi-year probability of default for each rating class which is basically presented in each migration matrix under column D representing default. A multi-year probability of Default for the rating class D is always 100% because it is assumed that its not possible to migrate to a higher, non-default class back again, once graded D.

In [13]:
# The 1-year Migration Matrix
markov_1y.style.background_gradient(cmap=cm).format("{:.2%}")

,AAA,AA,A,BBB,BB,B,CCC,D
AAA,87.09%,9.05%,0.52%,0.05%,0.10%,0.03%,0.05%,3.11%
AA,0.47%,87.42%,7.64%,0.46%,0.05%,0.06%,0.02%,3.88%
A,0.02%,1.54%,88.96%,4.86%,0.25%,0.10%,0.01%,4.26%
BBB,0.00%,0.08%,3.13%,86.95%,3.38%,0.40%,0.09%,5.97%
BB,0.01%,0.02%,0.06%,4.50%,78.30%,6.50%,0.53%,10.08%
B,0.00%,0.02%,0.06%,0.15%,4.50%,74.82%,4.81%,15.64%
CCC,0.00%,0.00%,0.08%,0.15%,0.46%,13.72%,44.74%,40.85%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


In [14]:
# The Migration Matrices for years 2 till 10th
# in order;
# 2-year Migration Matrix
# 3-year Migration Matrix
# 4-year Migration Matrix
# ...
dfs = [markov_2y, markov_3y, markov_4y, markov_5y, markov_6y, markov_7y, markov_8y, markov_9y, markov_10y]
for i in dfs: 
    display(i.style.background_gradient(cmap=cm).format("{:.2%}"))

,AAA,AA,A,BBB,BB,B,CCC,D
AAA,75.89%,15.80%,1.61%,0.16%,0.17%,0.07%,0.07%,6.23%
AA,0.82%,76.58%,13.49%,1.18%,0.12%,0.11%,0.03%,7.66%
A,0.04%,2.72%,79.41%,8.57%,0.59%,0.20%,0.02%,8.45%
BBB,0.00%,0.19%,5.51%,75.91%,5.61%,0.88%,0.16%,11.74%
BB,0.02%,0.04%,0.25%,7.45%,61.76%,10.04%,0.97%,19.48%
B,0.00%,0.03%,0.11%,0.46%,6.92%,56.93%,5.77%,29.77%
CCC,0.00%,0.00%,0.12%,0.24%,1.19%,16.43%,20.68%,61.33%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


,AAA,AA,A,BBB,BB,B,CCC,D
AAA,66.17%,20.71%,3.04%,0.33%,0.23%,0.11%,0.08%,9.34%
AA,1.08%,67.23%,17.89%,2.04%,0.21%,0.16%,0.04%,11.35%
A,0.07%,3.61%,71.12%,11.35%,0.96%,0.31%,0.04%,12.55%
BBB,0.00%,0.31%,7.30%,66.53%,7.01%,1.36%,0.21%,17.28%
BB,0.02%,0.06%,0.50%,9.29%,49.06%,11.69%,1.25%,28.13%
B,0.00%,0.04%,0.16%,0.81%,8.02%,43.84%,5.36%,41.77%
CCC,0.00%,0.01%,0.14%,0.33%,1.77%,15.21%,10.05%,72.49%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


,AAA,AA,A,BBB,BB,B,CCC,D
AAA,57.72%,24.14%,4.64%,0.58%,0.28%,0.14%,0.08%,12.42%
AA,1.26%,59.15%,21.13%,2.96%,0.32%,0.21%,0.04%,14.93%
A,0.09%,4.27%,63.90%,13.38%,1.33%,0.42%,0.06%,16.56%
BBB,0.01%,0.44%,8.61%,58.52%,7.82%,1.77%,0.26%,22.58%
BB,0.02%,0.08%,0.78%,10.33%,39.26%,12.15%,1.39%,35.99%
B,0.00%,0.05%,0.21%,1.14%,8.31%,34.06%,4.55%,51.68%
CCC,0.00%,0.01%,0.16%,0.41%,2.13%,12.88%,5.24%,79.18%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


,AAA,AA,A,BBB,BB,B,CCC,D
AAA,50.38%,26.40%,6.29%,0.88%,0.33%,0.17%,0.08%,15.47%
AA,1.38%,52.15%,23.41%,3.89%,0.45%,0.25%,0.05%,18.43%
A,0.11%,4.74%,57.59%,14.82%,1.67%,0.53%,0.07%,20.47%
BBB,0.01%,0.57%,9.53%,51.66%,8.20%,2.11%,0.30%,27.63%
BB,0.02%,0.10%,1.05%,10.80%,31.65%,11.87%,1.42%,43.07%
B,0.00%,0.06%,0.25%,1.44%,8.10%,26.65%,3.72%,59.78%
CCC,0.00%,0.02%,0.17%,0.49%,2.29%,10.49%,2.97%,83.58%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


,AAA,AA,A,BBB,BB,B,CCC,D
AAA,44.01%,27.73%,7.90%,1.23%,0.38%,0.20%,0.08%,18.47%
AA,1.45%,46.08%,24.94%,4.78%,0.58%,0.29%,0.05%,21.83%
A,0.13%,5.05%,52.06%,15.79%,1.98%,0.63%,0.09%,24.27%
BBB,0.01%,0.69%,10.14%,45.75%,8.29%,2.37%,0.32%,32.42%
BB,0.03%,0.13%,1.31%,10.89%,25.69%,11.18%,1.39%,49.39%
B,0.00%,0.06%,0.30%,1.67%,7.61%,20.99%,2.99%,66.38%
CCC,0.00%,0.02%,0.17%,0.55%,2.29%,8.41%,1.85%,86.70%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


,AAA,AA,A,BBB,BB,B,CCC,D
AAA,38.46%,28.35%,9.42%,1.62%,0.42%,0.23%,0.08%,21.43%
AA,1.48%,40.80%,25.87%,5.61%,0.71%,0.34%,0.06%,25.13%
A,0.15%,5.24%,47.20%,16.37%,2.25%,0.73%,0.10%,27.97%
BBB,0.02%,0.80%,10.51%,40.66%,8.17%,2.55%,0.35%,36.95%
BB,0.03%,0.15%,1.54%,10.71%,21.00%,10.27%,1.30%,55.01%
B,0.00%,0.07%,0.34%,1.85%,6.97%,16.61%,2.39%,71.77%
CCC,0.00%,0.02%,0.18%,0.61%,2.20%,6.70%,1.24%,89.04%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


,AAA,AA,A,BBB,BB,B,CCC,D
AAA,33.63%,28.41%,10.79%,2.04%,0.47%,0.25%,0.07%,24.33%
AA,1.49%,36.21%,26.31%,6.35%,0.85%,0.38%,0.06%,28.35%
A,0.16%,5.33%,42.90%,16.65%,2.47%,0.82%,0.11%,31.55%
BBB,0.02%,0.89%,10.69%,36.24%,7.92%,2.66%,0.36%,41.22%
BB,0.03%,0.17%,1.74%,10.35%,17.27%,9.27%,1.20%,59.98%
B,0.00%,0.07%,0.38%,1.96%,6.28%,13.22%,1.91%,76.17%
CCC,0.00%,0.03%,0.19%,0.65%,2.05%,5.33%,0.89%,90.87%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


,AAA,AA,A,BBB,BB,B,CCC,D
AAA,29.42%,28.05%,12.01%,2.47%,0.53%,0.28%,0.07%,27.18%
AA,1.47%,32.20%,26.38%,7.01%,0.98%,0.42%,0.07%,31.47%
A,0.18%,5.35%,39.10%,16.70%,2.64%,0.90%,0.12%,35.00%
BBB,0.03%,0.98%,10.72%,32.39%,7.57%,2.71%,0.36%,45.23%
BB,0.03%,0.19%,1.90%,9.88%,14.30%,8.27%,1.08%,64.36%
B,0.01%,0.07%,0.42%,2.03%,5.59%,10.57%,1.52%,79.79%
CCC,0.00%,0.03%,0.20%,0.68%,1.87%,4.24%,0.67%,92.32%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


,AAA,AA,A,BBB,BB,B,CCC,D
AAA,25.76%,27.37%,13.06%,2.90%,0.58%,0.30%,0.07%,29.97%
AA,1.44%,28.69%,26.16%,7.57%,1.11%,0.46%,0.07%,34.50%
A,0.19%,5.31%,35.72%,16.57%,2.78%,0.97%,0.13%,38.34%
BBB,0.03%,1.05%,10.63%,29.04%,7.17%,2.71%,0.36%,49.00%
BB,0.02%,0.21%,2.03%,9.34%,11.91%,7.31%,0.97%,68.21%
B,0.01%,0.08%,0.45%,2.06%,4.93%,8.49%,1.22%,82.77%
CCC,0.00%,0.03%,0.20%,0.69%,1.68%,3.39%,0.51%,93.49%
D,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%


As mentioned before, using data form columns D from each year migration matrix, the below cumulative multi-year Probability of Default table is created. It can be seen, especially on the below Figure 4 plot of the probabilities that they possess a tendency of monotonic increase and that they are non-overlapping. Left side Y axis represents the percentage (PD) as proportion of 1.

In [15]:
# Using data from columns "D" from yeach year Migration Matrix, 
# The below Cummulative Multi-year Probability of Default (CMPD) table is created
MPD = pd.DataFrame()
i = 1
for year in [markov_1y, markov_2y, markov_3y, markov_4y, markov_5y, markov_6y, markov_7y, markov_8y, markov_9y, markov_10y]:
    MPD[f"{i}-year Default"] = year.loc[:, 'D'].values
    i+=1

MPD = MPD.transpose()

# Adding the column headers as rating classes
MPD.columns = markov_1y.columns
MPD = MPD.iloc[:,:-1]

# Adjusting the visual aspects of the table.
Cumulative_Multi_year_probability_of_Default = MPD.style.background_gradient(cmap=cm).format("{:.2%}")

Cumulative_Multi_year_probability_of_Default

,AAA,AA,A,BBB,BB,B,CCC
1-year Default,3.11%,3.88%,4.26%,5.97%,10.08%,15.64%,40.85%
2-year Default,6.23%,7.66%,8.45%,11.74%,19.48%,29.77%,61.33%
3-year Default,9.34%,11.35%,12.55%,17.28%,28.13%,41.77%,72.49%
4-year Default,12.42%,14.93%,16.56%,22.58%,35.99%,51.68%,79.18%
5-year Default,15.47%,18.43%,20.47%,27.63%,43.07%,59.78%,83.58%
6-year Default,18.47%,21.83%,24.27%,32.42%,49.39%,66.38%,86.70%
7-year Default,21.43%,25.13%,27.97%,36.95%,55.01%,71.77%,89.04%
8-year Default,24.33%,28.35%,31.55%,41.22%,59.98%,76.17%,90.87%
9-year Default,27.18%,31.47%,35.00%,45.23%,64.36%,79.79%,92.32%
10-year Default,29.97%,34.50%,38.34%,49.00%,68.21%,82.77%,93.49%


In [16]:
# Saving the Data Frame for further creation of PIT-PD model. Saved as csv 
MPD.to_csv('cum.csv')

In [17]:
# Creating a plot of MPD using plotly.express
import plotly.express as px


df = pd.DataFrame(MPD)

fig = px.line(df)
fig.update_layout(title='Cumulative Multi-year Probability of Default', autosize=False,
                  width=600, height=600,
                  margin=dict(l=65, r=50, t=90),
                  xaxis_title="Years",
                  yaxis_title="Cumulative Multi-year Probability of Default")

fig.update_traces(mode='markers+lines')
fig.show()

A further step that can be taken is the calculation the Marginal Probabilities of Default using the Cumulative multi-year PDs (MPD) obtained before. This will allow, and also simplify, the calculation of Conditional Probabilities of Default.

In [18]:
# Calculation of the Marginal Probabilities of Deafult,
    # using Cumulative Multi-year PDs obtained before

# This will allow,and also simplify,
    # the calculation of Conditional Probabilities of Default

PDMU = pd.DataFrame(MPD.iloc[1:].values - MPD.iloc[:-1].values, columns= MPD.columns)

# Through-The-Cycle Conditional PDs
x = 1 - MPD.iloc[:-1].values 
SPD  = pd.DataFrame(PDMU/x, columns=MPD.columns)
SPD = SPD.set_index(MPD.index[1:])
SPD.style.format("{:.2%}") 
SPD.loc['1-year Default'] = MPD.iloc[0,:].to_list() 

SPD.reindex(MPD.index).style.background_gradient(cmap=cm).format("{:.2%}")

,AAA,AA,A,BBB,BB,B,CCC
1-year Default,3.11%,3.88%,4.26%,5.97%,10.08%,15.64%,40.85%
2-year Default,3.22%,3.93%,4.37%,6.13%,10.45%,16.75%,34.63%
3-year Default,3.31%,3.99%,4.48%,6.28%,10.74%,17.08%,28.85%
4-year Default,3.40%,4.05%,4.59%,6.41%,10.94%,17.02%,24.31%
5-year Default,3.48%,4.11%,4.69%,6.52%,11.06%,16.77%,21.13%
6-year Default,3.55%,4.17%,4.79%,6.62%,11.11%,16.42%,19.02%
7-year Default,3.63%,4.23%,4.88%,6.70%,11.10%,16.02%,17.61%
8-year Default,3.70%,4.29%,4.97%,6.77%,11.04%,15.60%,16.63%
9-year Default,3.76%,4.36%,5.05%,6.83%,10.94%,15.17%,15.89%
10-year Default,3.83%,4.42%,5.13%,6.88%,10.82%,14.75%,15.29%


In [19]:
# Saving the Data Frame for further creation of PIT-PD model. Saved as csv 
SPD.to_csv('cond.csv')

In [20]:
PDMU = PDMU.set_index(MPD.index[1:])
PDMU.style.format("{:.2%}") 
PDMU.loc['1-year Default'] = MPD.iloc[0,:].to_list() 

# Through-The-Cycle Marginal PD
PDMU.reindex(MPD.index).style.background_gradient(cmap=cm).format("{:.2%}")


,AAA,AA,A,BBB,BB,B,CCC
1-year Default,3.11%,3.88%,4.26%,5.97%,10.08%,15.64%,40.85%
2-year Default,3.12%,3.78%,4.19%,5.77%,9.40%,14.13%,20.48%
3-year Default,3.11%,3.68%,4.10%,5.54%,8.65%,11.99%,11.16%
4-year Default,3.08%,3.59%,4.01%,5.30%,7.86%,9.91%,6.69%
5-year Default,3.05%,3.49%,3.91%,5.05%,7.08%,8.10%,4.40%
6-year Default,3.00%,3.40%,3.81%,4.79%,6.32%,6.60%,3.12%
7-year Default,2.96%,3.31%,3.69%,4.53%,5.62%,5.39%,2.34%
8-year Default,2.90%,3.21%,3.58%,4.27%,4.97%,4.40%,1.82%
9-year Default,2.85%,3.12%,3.46%,4.01%,4.38%,3.62%,1.45%
10-year Default,2.79%,3.03%,3.33%,3.77%,3.86%,2.98%,1.17%


In [21]:
# Final outcome can be presented in plot.
# To do that, the plotly express is used. Style is matching previous plot

df1 = pd.DataFrame(SPD.reindex(MPD.index))

fig = px.line(df1)
fig.update_layout(title='TTC Conditional Probability of Default', autosize=False,
                    width=600, height=600,
                    margin=dict(l=65, r=50, b=65, t=90),
                    xaxis_title="Years",
                    yaxis_title="TTC Conditional Probability of Default")

fig.update_traces(mode='markers+lines')
fig.show()

The above graph represents the TTC Conditional Probability of Default. It can be observed that the CCC rating has constantly decreasing it’s probably of default with the relatively high starting point. It is due to being the last non-default rating class before the actual default and with the time, it can be assumed that less and less CCC rated credits are left in the pool. All other rating classes possess a very slight upward trend. All PDs are non-overlapping.